<a href="https://colab.research.google.com/github/agnxsh/bert-baseline/blob/main/bert_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [9]:
text = "2023 will be the greatesr year for all of us!"
encoding = tokenizer.encode_plus(text, add_special_tokens = True, truncation = True, padding = "max_length",
                                return_attention_mask = True, return_tensors="pt")

#encode_plus returns a dictionary of values, other than just a list of values, because 
#encode_plus can return various types information, such as token_ids, attention_masks, etc, 
#hence the best thing to do in this case is to make it a dictionary, so that the individual 
#attributes are easily accessible

In [10]:
encoding

{'input_ids': tensor([[  101, 16798,  2509,  2097,  2022,  1996,  2307,  2229,  2099,  2095,
          2005,  2035,  1997,  2149,   999,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [11]:
input = encoding["input_ids"][0]
attention_mask = encoding["attention_mask"][0]
attention_mask

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [12]:
tokenizer.mask_token

'[MASK]'

#Masked Language Modelling -> Basically fill in the blanks with the most appropriate word xD

In [29]:
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased", return_dict=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
#logits are the prediction scores of the Language Modlling Head of the ouput of the BERT model, just before the softmax activation function,
#these logits are more like scoring system for ensuring which word in the vocabulary is the most appropriate.
text = " the Opera House in Australia is in, "+tokenizer.mask_token + " city"

In [31]:
input = tokenizer.encode_plus(text, return_tensors="pt")

In [32]:
mask_index = torch.where(input["input_ids"][0]==tokenizer.mask_token_id)

In [33]:
output =  model(**input)
logits = output.logits

In [34]:
softmax = F.softmax(logits, dim=-1)
#getting a probabilistic value
mask_word = softmax[0, mask_index, :]

In [35]:
top_10 = torch.topk(mask_word, 10, dim=1)[1][0]
top_10.shape
top_10.dim

<function Tensor.dim>

In [36]:
for token in top_10:
  word=  tokenizer.decode([token])
  new_sentence = text.replace(tokenizer.mask_token, word)
  print(new_sentence)

 the Opera House in Australia is in, sydney city
 the Opera House in Australia is in, melbourne city
 the Opera House in Australia is in, brisbane city
 the Opera House in Australia is in, adelaide city
 the Opera House in Australia is in, the city
 the Opera House in Australia is in, canberra city
 the Opera House in Australia is in, auckland city
 the Opera House in Australia is in, hobart city
 the Opera House in Australia is in, griffith city
 the Opera House in Australia is in, hume city
